In [143]:
#auto reload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
from  prep_data import load_data
import torch

In [145]:
(train_data, train_dataloader), (val_data, val_dataloader), (bow_en, bow_fr) = load_data(10000,1000)

c:\Users\Alhus\.conda\envs\projetmla\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [146]:
for x in train_dataloader:
    print(x["english"]["idx"])
    break

tensor([[   30,    25,     3,  ..., 30000, 30000, 30000],
        [  223, 30000, 30000,  ...,    10,     0, 30000],
        [  465, 30000,    11,  ...,    89,   808,     1],
        ...,
        [  126,     1,     0,  ..., 30000, 30000, 30000],
        [ 1676, 30000,    13,  ..., 30000, 30000, 30000],
        [   13,    85,    73,  ..., 30000, 30000, 30000]], dtype=torch.int16)


In [147]:
def idx_to_word(idx: torch.Tensor, vocab: List):
    return " ".join(list(vocab[idx[(idx < len(vocab))]]))

In [148]:
idx_to_word(torch.tensor([1,2,3,4,5,6,7,8,9,10,30000,30000,30000,30000,30000,30000,30000,30000]), data['bow']['english'])

'of and to a in for is on that by'

In [149]:
def to_vector(ids, vocab_size):
    vecs = torch.zeros(len(ids), vocab_size)
    vecs[range(len(ids)), ids] = 1
    return vecs

class to_vec:
    def __init__(self, tor, vocab_size, lang):
        self.tor = tor
        self.vocab_size = vocab_size
        self.lang = lang

    
    def __call__(self, ids):
        ids = self.tor.tensor(ids["ids_{}".format(self.lang)])
        vecs = self.tor.zeros(len(ids), self.vocab_size)
        vecs[range(len(ids)), ids] = 1
        return vecs

BLEU SCORE

1-EXISTANT NLTK

In [150]:
import nltk 
hypothesis=['Le','chat','se','trouve','sur','le','toit']
reference=['Le','chat','se','trouve','sur','le','toit','bla']
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis) 
print(BLEUscore)


0.8668778997501817


2-SCORE PERSONNALISE

In [21]:
import collections
import math

def ngram_precision(candidate, reference, n):
    candidate_ngrams = [tuple(candidate[i:i+n]) for i in range(len(candidate)-n+1)]
    reference_ngrams = [tuple(reference[i:i+n]) for i in range(len(reference)-n+1)]

    candidate_ngram_counts = collections.Counter(candidate_ngrams)
    reference_ngram_counts = collections.Counter(reference_ngrams)

    overlap_count = sum((candidate_ngram_counts & reference_ngram_counts).values())
    total_count = max(1, len(candidate_ngrams))

    precision = overlap_count
    return precision

def bleu_score(candidate, references, weights):
    precisions = []

    for i in range(len(weights)):
        n = i + 1
        precision = ngram_precision(candidate, references, n)
        precisions.append(precision)

    # geometric_mean
    bleu= math.exp(sum(weights[i] * math.log(p) for i, p in enumerate(precisions)))

    # brevity_penalty = min(1, len(candidate) / max(len(reference) for reference in references))
    # bleu = brevity_penalty * geometric_mean
    
    return bleu

# Exemple d'utilisation
reference = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
candidate = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']

# candidate=['Le','chat','se','trou','s','le','to','bli']
# reference=['Le','chat','se','trouve','sur','le','toit']

weights = [0.25,0.25,0.25] # You can adjust the weights based on your preference

score = bleu_score(candidate, reference, weights)
print("BLEU Score:", score)


BLEU Score: 4.738137220537587


In [30]:
import collections

def compute_bleu_score(reference, candidate, n=4):

    # Calcul des n-grammes pour la référence et le candidat
    reference_ngrams = [tuple(reference[i:i + n]) for i in range(len(reference) - n + 1)]
    candidate_ngrams = [tuple(candidate[i:i + n]) for i in range(len(candidate) - n + 1)]

    # Comptage des n-grammes dans la référence
    reference_ngram_counts = collections.Counter(reference_ngrams)

    # Comptage des n-grammes dans le candidat qui correspondent à la référence
    matching_ngram_counts = sum(min(reference_ngram_counts[ngram], candidate_ngrams.count(ngram)) for ngram in set(candidate_ngrams))

    # Calcul du score BLEU
    precision = matching_ngram_counts 

    return precision 

# Exemple d'utilisation
reference = "the quick brown fox jumped over the lazy dog hey"
candidate = "the quick brown fox jumped over the lazy dog hey"

# reference = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
# candidate = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']

score_bleu = compute_bleu_score(reference, candidate)
print("Score BLEU:", score_bleu)


Score BLEU: 45


In [34]:
pip install torch

Note: you may need to restart the kernel to use updated packages.Collecting torch
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/192.3 MB 640.0 kB/s eta 0:05:01
   ---------------------------------------- 0.6/192.3 MB 5.4 MB/s eta 0:00:36
   ---------------------------------------- 1.8/192.3 MB 11.5 MB/s eta 0:00:17
    --------------------------------------- 2.9/192.3 MB 14.3 MB/s eta 0:00:14
    --------------------------------------- 4.2/192.3 MB 16.7 MB/s eta 0:00:12
   - -------------------------------------- 5.2/192.3 MB 17.6 MB/s eta 0:00:11
   - -------------------------------------- 6.2/192.3 MB 18.0 MB/s eta 0:00:11
   - -------------------------------------- 7.4/192.3 MB 19.7 MB/s et

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim

# Exemple d'un modèle de langage simple pour l'illustration
class SimpleLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(SimpleLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        logits = self.linear(lstm_out)
        return logits

# Fonction pour calculer la NLL moyenne
def compute_average_nll(model, dataloader, criterion):
    total_loss = 0
    total_tokens = 0

    model.eval()  # Mettre le modèle en mode évaluation

    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            total_loss += loss.item()
            total_tokens += targets.numel()

    average_nll = total_loss / total_tokens

    return average_nll

# Exemple d'utilisation
vocab_size = 10000
embedding_dim = 100
hidden_dim = 128
model = SimpleLanguageModel(vocab_size, embedding_dim, hidden_dim)

# Supposons que vous ayez un DataLoader nommé 'train_dataloader' pour l'ensemble d'entraînement
# et un DataLoader nommé 'dev_dataloader' pour l'ensemble de développement
train_criterion = nn.CrossEntropyLoss()  # Vous pouvez utiliser d'autres critères en fonction de votre modèle
train_average_nll = compute_average_nll(model, train_dataloader, train_criterion)

dev_criterion = nn.CrossEntropyLoss()  # Vous pouvez utiliser d'autres critères en fonction de votre modèle
dev_average_nll = compute_average_nll(model, dev_dataloader, dev_criterion)

print("Average NLL on training set:", train_average_nll)
print("Average NLL on development set:", dev_average_nll)


NameError: name 'train_dataloader' is not defined

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Exemple de dataset
class LanguageDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx])

# Génération de données factices
train_data = [
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    # Ajoutez d'autres exemples d'entraînement ici
]

dev_data = [
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    # Ajoutez d'autres exemples de développement ici
]

# Création des datasets
train_dataset = LanguageDataset(train_data)
dev_dataset = LanguageDataset(dev_data)

# Création des DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=2, shuffle=False)

# Exemple d'utilisation du DataLoader
for inputs, targets in train_dataloader:
    print("Inputs:", inputs)
    print("Targets:", targets)
    break

# Exemple d'un modèle de langage simple pour l'illustration
class SimpleLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(SimpleLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        logits = self.linear(lstm_out)
        return logits

# Fonction pour calculer la NLL moyenne
def compute_average_nll(model, dataloader, criterion):
    total_loss = 0
    total_tokens = 0

    model.eval()  # Mettre le modèle en mode évaluation

    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            total_loss += loss.item()
            total_tokens += targets.numel()

    average_nll = total_loss  #/ total_tokens

    return average_nll

# Exemple d'utilisation
vocab_size = 10000
embedding_dim = 100
hidden_dim = 128
model = SimpleLanguageModel(vocab_size, embedding_dim, hidden_dim)

# Supposons que vous ayez un DataLoader nommé 'train_dataloader' pour l'ensemble d'entraînement
# et un DataLoader nommé 'dev_dataloader' pour l'ensemble de développement
train_criterion = nn.CrossEntropyLoss()  # Vous pouvez utiliser d'autres critères en fonction de votre modèle
dev_criterion = nn.CrossEntropyLoss()  # Vous pouvez utiliser d'autres critères en fonction de votre modèle

train_average_nll = compute_average_nll(model, train_dataloader, train_criterion)
dev_average_nll = compute_average_nll(model, dev_dataloader, dev_criterion)

print("Average NLL on training set:", train_average_nll)
print("Average NLL on development set:", dev_average_nll)


Inputs: tensor([1, 2, 3, 4, 5])
Targets: tensor([ 6,  7,  8,  9, 10])
Average NLL on training set: 9.161561965942383
Average NLL on development set: 9.16692066192627


In [39]:
import torch
import torch.nn.functional as F

def calculate_nll(sentences, lengths, model, device):
    """
    Calculate Negative Log-Likelihood (NLL) for a set of sentences.

    Args:
    - sentences: List of tensorized sentences
    - lengths: List of sentence lengths
    - model: PyTorch model for language modeling
    - device: Device on which to perform the calculations

    Returns:
    - nll: Average Negative Log-Likelihood
    """

    model.eval()  # Set the model to evaluation mode
    nll_sum = 0.0
    total_tokens = 0

    with torch.no_grad():
        for i in range(len(sentences)):
            input_sentence = sentences[i].to(device)
            target_sentence = input_sentence[:, 1:].contiguous()  # Shift the target by one position
            input_lengths = lengths[i]

            output = model(input_sentence[:, :-1], input_lengths)  # Model prediction
            output_flattened = output.view(-1, output.size(-1))
            target_flattened = target_sentence.view(-1)

            # Mask out padding tokens
            mask = (target_flattened != 0)  # Assuming 0 is the padding token index
            output_masked = output_flattened[mask]
            target_masked = target_flattened[mask]

            nll_sum += F.cross_entropy(output_masked, target_masked, ignore_index=0, reduction='sum').item()
            total_tokens += target_masked.size(0)

    nll = nll_sum / total_tokens

    return nll

# Example usage:
# Assume you have a PyTorch model called 'language_model'
# and tokenized sentences in 'train_sentences' and 'dev_sentences'
# with corresponding lengths in 'train_lengths' and 'dev_lengths'

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# nll_train = calculate_nll(train_sentences, train_lengths, language_model, device)
# nll_dev = calculate_nll(dev_sentences, dev_lengths, language_model, device)

# print(f"Train NLL: {nll_train}")
# print(f"Dev NLL: {nll_dev}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Modèle de langage fictif (à titre d'exemple)
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)
        output, _ = self.rnn(packed)
        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        output = self.fc(output)
        return output

# Exemple de données
train_sentences = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8, 9]])
train_lengths = [4, 5]

dev_sentences = torch.tensor([[1, 2, 3], [4, 5, 6, 7]])
dev_lengths = [3, 4]

# Hyperparamètres fictifs
vocab_size = 10
embedding_dim = 16
hidden_dim = 32
learning_rate = 0.001

# Instancier le modèle
model = LanguageModel(vocab_size, embedding_dim, hidden_dim)

# Fonction de perte et optimiseur
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignorer les indices de rembourrage
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Entraînement fictif (c'est un exemple simple, pas un entraînement complet)
epochs = 10
for epoch in range(epochs):
    model.train()
    for i in range(len(train_sentences)):
        input_sentence = train_sentences[i].unsqueeze(0)
        input_length = train_lengths[i]

        target_sentence = input_sentence[:, 1:].contiguous()
        output = model(input_sentence[:, :-1], [input_length])

        optimizer.zero_grad()
        loss = criterion(output.view(-1, vocab_size), target_sentence.view(-1))
        loss.backward()
        optimizer.step()

# Calcul de NLL pour les ensembles d'entraînement et de développement
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nll_train = calculate_nll(train_sentences, train_lengths, model, device)
nll_dev = calculate_nll(dev_sentences, dev_lengths, model, device)

print(f"Train NLL: {nll_train}")
print(f"Dev NLL: {nll_dev}")
